<a href="https://colab.research.google.com/github/kevinCortial/graphlearning/blob/main/classification_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> Graph Learning on Product Space </center></h1>

<center> Kévin CORTIAL et Taoufik JARMOUNI &nbsp;&nbsp;&nbsp; | &nbsp;&nbsp;&nbsp; Open Studio UCA IP / Atlas des Synergies Productives &nbsp;&nbsp;&nbsp; </center>

In [ ]:
pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 3.4 MB/s 
     |████████████████████████████████| 291 kB 43.8 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
pip install dgllife

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 4.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [ ]:
from google.colab import drive

#This cell allows us to grab our needed resources from Google Drive directly

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

print(g.ndata)
print(g)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Graph(num_nodes=270

g.subgraph() déjà essayé (par contre je ne suis pas sûr d'avoir tout essayé)

nx.weakly_connected_components(g)

dgl.data.utils.Subset()

dgl.distributed.dist_graph.node_split()

In [ ]:
from dgl.nn import GraphConv
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from dgl.data.utils import split_dataset, Subset
from dgllife.utils import ConsecutiveSplitter

g = nx.read_gpickle("/content/drive/MyDrive/Open_Studio/product_graphWithData4int.gpickle")
#categorie = list(g.nodes(data="categorie_macro"))

RDKit is not installed, which is required for utils related to cheminformatics


In [ ]:
from dgl.nn import GraphConv
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from dgl.data.utils import split_dataset, Subset
from dgllife.utils import ConsecutiveSplitter

g = nx.read_gpickle("/content/drive/MyDrive/Open_Studio/product_graphWithData4int.gpickle")
#categorie = list(g.nodes(data="categorie_macro"))

#print(categorie)
#tpl_categorie = list(zip(*categorie))[1]
#print(tpl_categorie)
#lst_categorie = list(tpl_categorie)
#print(lst_categorie)
g = dgl.from_networkx(g, node_attrs=['data', 'categorie_macro'])

test = ConsecutiveSplitter.train_val_test_split(dataset, frac_train=0.8, frac_val=0.1, frac_test=0.1)

print(test)


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['data']
    labels = g.ndata['categorie_macro']
    #lst_int_categorie = [int(x) for x in lst_categorie]
    #pt_tensor_from_list = torch.FloatTensor(lst_int_categorie)

    #labels = pt_tensor_from_list
    #liste pytorch des ID catégorie
    '''
    train_mask = g.ndata['train_mask']
    print(train_mask)
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    '''
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        '''
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])


        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
        '''

        # Backward
        optimizer.zero_grad()
        '''
        loss.backward()'''
        optimizer.step()

        '''
        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))'''
            
        if e % 5 == 0:
            print('In epoch {}, (best {:.3f}), (best {:.3f})'.format(
                e, best_val_acc, best_test_acc))
            

# Create the model with given dimensions
model = GCN(g.ndata['data'].shape[1], 16, dataset.num_classes)
train(g, model)

[<dgl.data.utils.Subset object at 0x7f0452f60650>, <dgl.data.utils.Subset object at 0x7f0452f60810>, <dgl.data.utils.Subset object at 0x7f0452f60990>]
In epoch 0, (best 0.000), (best 0.000)
In epoch 5, (best 0.000), (best 0.000)
In epoch 10, (best 0.000), (best 0.000)
In epoch 15, (best 0.000), (best 0.000)
In epoch 20, (best 0.000), (best 0.000)
In epoch 25, (best 0.000), (best 0.000)
In epoch 30, (best 0.000), (best 0.000)
In epoch 35, (best 0.000), (best 0.000)
In epoch 40, (best 0.000), (best 0.000)
In epoch 45, (best 0.000), (best 0.000)
In epoch 50, (best 0.000), (best 0.000)
In epoch 55, (best 0.000), (best 0.000)
In epoch 60, (best 0.000), (best 0.000)
In epoch 65, (best 0.000), (best 0.000)
In epoch 70, (best 0.000), (best 0.000)
In epoch 75, (best 0.000), (best 0.000)
In epoch 80, (best 0.000), (best 0.000)
In epoch 85, (best 0.000), (best 0.000)
In epoch 90, (best 0.000), (best 0.000)
In epoch 95, (best 0.000), (best 0.000)
